In [1]:
import re
import requests
import numpy as np
import pandas as pd
from time import sleep
from random import randint
from bs4 import BeautifulSoup

In [2]:
Job_Title_Name = []
Company_Name = []
Experience_Required = []
Salary = []
Location = []
Job_Description = []
Key_Skills = []

In [3]:
for i in range(1, 21):
    url = "https://www.timesjobs.com/candidate/job-search.html?from=submit&actualTxtKeywords=AI%20Engineer&searchBy=0&rdoOperator=OR&searchType=personalizedSearch&txtLocation=India&luceneResultSize=25&postWeek=60&txtKeywords=ai%20engineer&pDate=I&sequence=" + str(i) + "&startPage=1"
    # print(url)
    sleep(randint(2,10))



    # Making a GET request
    r = requests.get(url)

    # check status code for response received
    # success code - 200
    # print(r)

    # print content of request
    htmlContent = r.content
    # print(htmlContent)



    # Parse the HTML
    soup = BeautifulSoup(htmlContent, 'html.parser')
    # print(soup.prettify)



    # HTML Tree Traversal

    # Get the Title of the HTML page
    title = soup.title
    # print(title.string)



    Job_Title = soup.find_all('h2')

    for h2 in Job_Title:
        A=h2.get_text()
        t1 = re.split(r'\n\n|\n\r\n', A)
        Job_Title_Name.append(t1[1])

    # print(Job_Title_Name)



    Company = soup.find_all('h3')
    Tem = []

    for Company_name in Company:
        C_Name=Company_name.get_text()
        t1 = C_Name.split('\r\n')
        
        Tem.append(t1)

    res = [ele for ele in Tem if ele != ['Discover your']]

    for Company_name_1 in res:
        Company_Name.append(Company_name_1[1])

    # print(Company_Name)



    Experience = soup.find_all('ul', attrs = {'class': 'top-jd-dtl clearfix'})

    for Exp in Experience:
        E=Exp.get_text()
        
        temp1 = E.split("card_travel")
        temp2 = temp1[1].splitlines()
        Experience_Required.append(temp2[0])

    # print(Experience_Required)



    Sal = soup.find_all('ul', attrs = {'class': 'top-jd-dtl clearfix'})

    for S in Sal:
        Sa=S.get_text()
        
        temp1 = Sa.split("\n")
        temp2 = temp1[2]
        if(temp2.__contains__('₹Rs')):
            Salary.append(temp2)
        else:
            Salary.append(np.nan)

    # print(Salary)



    Loc = soup.find_all('ul', attrs = {'class': 'top-jd-dtl clearfix'})
    
    for L in Loc:
        Lo=L.get_text()
        
        temp3 = Lo.splitlines()
        if(temp3[4] != 'location_on'):
            Location.append(temp3[4])
        else:
            Location.append(temp3[5])

    # print(Location)



    Job_Desc = soup.find_all('ul', attrs = {'class': 'list-job-dtl clearfix'})

    for Desc in Job_Desc:
        D=Desc.get_text()
        
        temp1 = D.split("\n\nJob Description:\r\n")
        temp2 = temp1[1].split("\n\n\nKeySkills:\n\r\n      \r\n          ")
        
        if(temp2[0].__contains__('... More Details')):
            temp3 = temp2[0].split("... More Details")
            Job_Description.append(temp3[0])
        else:
            Job_Description.append(temp2[0])

    # print(Job_Description)



    Job_Desc = soup.find_all('ul', attrs = {'class': 'list-job-dtl clearfix'})

    for Desc in Job_Desc:
        D=Desc.get_text()
        
        temp1 = re.split(r'\n\n\nKeySkills:\n\r\n      \r\n          |\n\n\nKeySkills:|\r\n        \r\n       \n\n', D)
        Key_Skills.append(temp1[1])
        
    # print(Key_Skills)

In [4]:
data = {
        'Sr. No.': [*range(1, len(Job_Title_Name)+1)], 
        'Job Title': Job_Title_Name, 
        'Company Name': Company_Name, 
        'Experience Required': Experience_Required, 
        'Salary': Salary, 
        'Location': Location, 
        'Job Description': Job_Description, 
        'Key Skills': Key_Skills
}

df = pd.DataFrame(data)
print(df.head(5))

   Sr. No.     Job Title                               Company Name  \
0        1  AI Engineer                          Intel Technologies   
1        2  AI Engineer                   Bloom Consulting Services   
2        3  AI Engineer               innovation incubator advisory   
3        4  AI Engineer                            Analytics Vidhya   
4        5  AI Engineer       ikomet Technology Solutions Pvt. Ltd.   

  Experience Required Salary                                    Location  \
0           5 - 8 yrs    NaN                       Bengaluru / Bangalore   
1           2 - 5 yrs    NaN                       Bengaluru / Bangalore   
2           0 - 3 yrs    NaN                          Thiruvananthapuram   
3           3 - 6 yrs    NaN  Thiruvananthapuram,  Bengaluru / Bangalore   
4           0 - 3 yrs    NaN                                     Chennai   

                                     Job Description  \
0  AI Engineer Job DescriptionIntel Emergent AI R...   
1  D

In [5]:
df.to_csv('Times_Jobs_Dataset.csv', index = False)